# **Assignment 1**


### Insert your details below. You should see a green checkmark.

In [25]:
# Input student details (replace these with actual input values)
student = {
    "name": "Shaaz Feerasta",  # Replace with your name
    "email": "feerasta@ualberta.ca",       # Replace with your email
    "ccid": "feerasta",              # Replace with your CCID
    "idnumber": 1704756,               # Replace with your ID number
}

In [26]:
# Define the default student structure
def_student = {
    "name": "NAME as in eclass",
    "email": "UofA Email",
    "ccid": "CCID",
    "idnumber": 0,
}

# Validation checks
assert set(def_student.keys()) == set(student.keys()),   "You don't have all the right entries! Make sure you have `name`, `email`, `ccid`, `idnumber`. ❌"
assert not any(value == "" for value in student.values()), "You haven't filled in all your details! No field should be empty. ❌"
assert all(isinstance(student[k], type(def_student[k])) for k in def_student),    "Your types seem to be off: `name::String`, `email::String`, `ccid::String`, `idnumber::Int`. ❌"
assert student["email"].endswith("@ualberta.ca"), "Your email must end with '@ualberta.ca'. ❌"

print(f"Welcome {student['name']}! ✅")



Welcome Shaaz Feerasta! ✅


# Preamble

- Loading Packages
- Generating Utilities

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats import multinomial
from numpy.random import default_rng
import plotly.express as px
from scipy.stats import invgauss


# Generalized Linear Models



In this assignment, you will implement two different Generalized Linear Models (GLMs) based on the derivations covered in class and in the notes.

As a warm-up—and a soft introduction to Python—you will extend our implementation of linear regression to:
1. Use an L2 regularizer.
2. Use the ADAM optimizer.

Our implementation of linear regression is simple: we use stochastic gradient descent (SGD) with a scalar fixed step size, where each update only uses one sample $(\mathbf{x}_i, y_i)$ (rather than a larger mini-batch):

$$
\mathbf{w}_{t+1} = \mathbf{w}_t - \eta (f_{\mathbf{w}_t}(\mathbf{x}_i) - y_i) \mathbf{x}_i
$$

After completing these two modifications to linear regression, you will implement your own multinomial logistic regression.

### Notes:
- In each part of the assignment, look for ❌ in certain cells. These denote tasks that you are expected to complete.
- Several tests are provided to help debug your implementation. **Do not modify these test cells.**

Keep in mind: Passing the provided tests does not guarantee correctness, as the tests are not comprehensive.


# **Linear Regression (warm up)**

## Synthetic Data

You will start by creating a synthetic dataset. The purpose of this is to create controlled situations to test your methods. We can create synthetic datasets that match the assumptions underlying a method, and so test our implementations. If they do not behave as we expect, this might indicate a bug (conceptually or in our code).

Here, you will first create synthetic data where the targets $y$ are generated by sampling from a Gaussian where the mean is a linear function $f$ of $\mathbf{x}$ with some fixed variance $\sigma^2$. This can be done programmatically by:  
1. Generating a random input vector $\mathbf{x}$,  
2. Sampling noise $\epsilon \sim \mathcal{N}(0, \sigma^2)$, and  
3. Setting the target to $y = f(\mathbf{x}) + \epsilon$.

We did not discuss how to generate $\mathbf{x}$, nor did we make assumptions about $p(\mathbf{x})$ in linear regression. However, to create a synthetic dataset, we need to decide what distribution to use. A simple choice is to generate these inputs randomly from Gaussians (just because we are so used to Gaussians).

To make the outcome more interesting on this synthetic data:
- We generate the inputs from an inverse Gaussian distribution.
- We also add dependencies between these inputs to observe their impact on linear regression.

This function that generates $\mathbf{x}$ is in `generate_features`.

Later, you will reason about the types of datasets where we expect L2 regularization to improve performance.

We have provided the function that generates $\mathbf{x}$. Your job is to specify a linear function $f$ for this synthetic data generation. We will also provide the code that generates the Gaussian noise to create the target $y$.


In [28]:
num_samples = 30000

# Convenient to have the number of features assigned to a variable
num_features = 15 + 1  # 15 base features + 1 added column of ones


In [29]:
def generate_features(rng, num_samples=30000):
    """
    Generate features according to an Inverse Gaussian distribution (μ = 0.1 and λ = 1).
    This function uses a funky distribution to help detect bugs later on. If a Normal
    distribution is used here, it might hide some bugs.

    Args:
        rng: A random number generator (e.g., np.random.default_rng()).
        num_samples: Number of samples to generate (default: 30000).

    Returns:
        X: A numpy array of generated features.
    """
    # Generate μs from a uniform distribution
    μs = rng.random(10)

    # Generate features using the Inverse Gaussian distribution
    X = np.column_stack([invgauss.rvs(mu=μs[i], scale=1, size=num_samples, random_state=rng) for i in range(10)])

    # Add some components which are linearly dependent
    X = np.column_stack([
        X,
        0.33 * X[:, 0] + 0.67 * X[:, 4],
        -X[:, 3],
        X[:, 2] - X[:, 7],
        X[:, 8] + 0.5 * X[:, 9],
        X[:, 5] - 0.5 * X[:, 1]
    ])

    # Append a column of ones to act as a bias unit
    bias = np.ones((num_samples, 1))
    X = np.hstack((X, bias))

    return X


In the below, specify the linear function that we will use to generate the synthetic data. We leave it up to you to pick this linear function, but there are a few restrictions:

1. Don't use $f(x) = c$ for some constant $c$.  
2. Make sure $f(x)$ depends on at least 2 columns of $X$.  
3. Your function should be single variate (1 output).


In [68]:
rng = np.random.default_rng(10)  # Random number generator with a fixed seed
W_true = None


w_temp = rng.normal(0, 0.2, size=(num_features, 1))

# TODO: Here you can modify w_temp. The weight W_true will be set to w_temp.
# This is the place where you get to specify the weights for the function f(X).
#### BEGIN SOLUTION
w_temp[0] = 0
w_temp[1] = 1
w_temp[2] = 2
#### END SOLUTION

W_true = w_temp

In [69]:
print(W_true.shape)

(16, 1)


In [70]:
# TODO: f should be a linear function that uses the weights W_true
def f(X, W_true):
    #### BEGIN SOLUTION
    return X @ W_true
    #### END SOLUTION


### Checkpoint: Is `f` Linear?

To test this question, we use two properties of linear functions:

1. Preserve vector addition: $f(\mathbf{a}) + f(\mathbf{b}) \approx f(\mathbf{a} + \mathbf{b})$
2. Preserve scalar multiplication: $c f(\mathbf{a}) \approx f(c \mathbf{a})$


In [71]:
# Check if f satisfies vector addition property
def __check_f_vec_addition():
    rng = np.random.default_rng(42)
    a = rng.uniform(-10, 10, (100, num_features))
    b = rng.normal(2, 3, (100, num_features))
    return np.allclose(f(a, W_true) + f(b, W_true), f(a + b, W_true))

# Check if f satisfies scalar multiplication property
def __check_f_scalar_mult():
    rng = np.random.default_rng(42)
    a = rng.uniform(-10, 10, (100, num_features))
    c = rng.uniform(-5, 5)
    return np.allclose(c * f(a, W_true), f(c * a, W_true))


# Local checks for each property
try:
    __local_check_f_vec_addition = __check_f_vec_addition()
except Exception:
    __local_check_f_vec_addition = False

try:
    __local_check_f_scalar_mult = __check_f_scalar_mult()
except Exception:
    __local_check_f_scalar_mult = False

__check_f_vec_addition()
# Print summary for checkpoint
print("Checkpoint: Is `f` Linear?")
assert __local_check_f_vec_addition, "1. Preserve vector addition: ❌"
print(f"1. Preserve vector addition: ✅")

assert __local_check_f_scalar_mult, "2. Preserve scalar multiplication: ❌"
print(f"2. Preserve scalar multiplication: ✅")


Checkpoint: Is `f` Linear?
1. Preserve vector addition: ✅
2. Preserve scalar multiplication: ✅


In [72]:
def generate_targets(X, rng, W_true):
    """
    Generate targets for the synthetic dataset.

    Parameters:
    - X: Feature matrix (numpy.ndarray)
    - rng: Random number generator (numpy.random.Generator)

    Returns:
    - Y: Targets for the dataset, generated as f(X) plus random noise
    """
    # Generate Y ~ Normal(f(X), 1)
    # Use the function `f(X)` to generate non-perturbed targets
    # Use `rng.normal` for random noise (mean=0, stddev=1)

    #### BEGIN SOLUTION
    Y = f(X, W_true) + rng.normal(0,1, f(X, W_true).shape)
    #### END SOLUTION

    return Y

Now we generate the data we are going to use for the Linear models!

If you have implemented the random perturbation correctly, you should see an approximately normal distribution below.


In [73]:
rng = np.random.default_rng(358)
X = generate_features(rng)
Y = generate_targets(X, rng, W_true)

# Assuming Y is a NumPy array
fig = px.histogram(
    x=Y[:, 0],
    nbins=80,
    opacity=0.7,
    title="Target Distribution",
    labels={"x": "Target Value", "y": "Frequency"}
)
fig.update_layout(
    xaxis_title="Target Value",
    yaxis_title="Frequency",
    bargap=0.1
)
fig.show()

# Linear Models
## Linear Models

Each model must have a consistent API.  
We start with a class which is a subtype of `Model`.  
Each subtype of `Model` must provide a `predict` function, which takes as input an instance of the model and a data matrix `X`, then returns a vector of predictions `y`.  
To update the weights of a model instance, you will need to implement the `update` function, which takes the model instance, an optimizer, a data matrix of inputs, and a data vector of responses.  
Note the difference in naming: the `update` function will **mutate** the weights belonging to the model instance.


In [74]:
# Define RMSE and L1Error functions

def RMSE(x_hat, x):
    print(x_hat.shape)
    print(x.shape)
    return np.sqrt(mean_squared_error(x_hat, x))

def L2Error(x_hat, x):
    return RMSE(x_hat, x)

def L1Error(x_hat, x):
    return np.linalg.norm(x_hat - x, 1)

# Create a base class for
class Model:
    pass

# Create a shared parent class for Optimizer
class Optimizer:
    pass

In [75]:
# Split data into training and testing
trainX = X[:25000, :]
testX = X[25000:, :]

trainY = Y[:25000, :]
testY = Y[25000:, :]

In [76]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(25000, 16)
(25000, 1)
(5000, 16)
(5000, 1)


### Simple Linear Regression

As mentioned above, this implementation of linear regression (LR) uses stochastic gradient descent (SGD) with one sample. It is implemented as follows: for one epoch, the dataset is randomly shuffled, and then iterated over in order, updating for each sample \(i\). A fixed scalar step size is used, rather than adaptive vector step sizes.

We opted for a simpler implementation of linear regression, where we do not use mini-batches. This implementation is simple to extend.


In [77]:
# Linear Model class
class LinearModel(Model):
    def __init__(self, in_dim, out_dim=1):
        self.W = np.zeros((in_dim, out_dim))

    def predict(self, X):
      return np.dot(X, self.W)



# Stochastic Gradient Descent Optimizer
class SGD(Optimizer):
    def __init__(self, alpha):
        self.alpha = alpha

def update(lm, sgd, x, y):
    # Make a prediction using the model's weights
    y_hat = lm.predict(x)
    #print(y_hat.shape)
    #print(y.shape)

    # Compute the error
    error = y_hat - y
    #print(error.shape, "error")

    # Compute the gradient (delta_W)
    delta_W = (error * x ).reshape(-1,1) # This will have shape (n_features, 1)
    #print((sgd.alpha * delta_W).shape, "weight update shape")

    # Update the weights by subtracting the gradient scaled by the learning rate
    lm.W  = lm.W - sgd.alpha * delta_W
   # print(lm.W.shape, "are we okay")



def train(m, sgd, X, Y):
    """
    Train the model using stochastic gradient descent (SGD) with the given data.

    Args:
        m: Model instance (e.g., LinearModel).
        sgd: Optimizer instance (e.g., SGD).
        X: Input data matrix (2D numpy array).
        Y: Target data matrix (2D numpy array).
    """
    # Set random seed for reproducibility
    np.random.seed(1)

    # Shuffle the indices

    indices = np.random.permutation(Y.shape[0])

    # Iterate over the shuffled data
    for idx in indices:
        # Grab a row of X
        x = X[idx, :]
        y = Y[idx, :]
        #print(x.shape,y.shape, "x and y for training")

        # Update the model using the optimizer, input x, and target y
        update(m, sgd, x, y)

def run_linear_model(X, Y, num_epochs=100):

    # Construct the model
    sgd = SGD(0.01)  # Learning rate of 0.01
    model = LinearModel(X.shape[1], 1)  # Create a model with input dimension X.shape[1]
    #print("weights shape:",model.W.shape)

    # Each call to train performs a single epoch
    for _ in range(num_epochs):
      #print(_)
      train(model, sgd, X, Y)

    return model

In [78]:
# Train the linear model
linear_model = run_linear_model(trainX, trainY, num_epochs=10)

# Compute RMSE on the test data
predictions = linear_model.predict(testX)
print("prediction shape",predictions.shape)

rmse_linear = RMSE(predictions, testY)
print("RMSE:", rmse_linear)


prediction shape (5000, 1)
(5000, 1)
(5000, 1)
RMSE: 1.0074355228732113


## Regularized LR


In this section, you will implement linear regression with L2 regularization (RLR). Modify the update rule for the linear regression solution given above to add L2 regularization. The update is:

$$
\mathbf{w}_{t+1} = \mathbf{w}_t - \eta \left( f_{\mathbf{w}_t}(\mathbf{x}_i) - y_i \right) \mathbf{x}_i - \eta \lambda  \mathbf{w}_t
$$

Note that technically in the notes $\lambda$ is scaled by the number of samples. However, here we have a fixed number of samples. We therefore keep it simpler here and just specify an appropriate constant for the regularization.


In [79]:
# We will get you started with the RegularizedModel interface
class RegularizedModel(Model):
    def __init__(self, in_dim, out_dim, lambda_):
        self.W = np.zeros((in_dim, out_dim))
        self.lambda_ = lambda_

    def predict(self, x):
       return np.dot(x, self.W)


Now we need to implement the update! function for the RegularizedModel! Below this function (which you need to implement) you should see a test cell. You can use this to test your code, but ask that you don't change the code in this cell. Doing so will result in point deductions!!!

In [80]:

def update(lm, sgd, x, y):
    # BEGIN SOLUTION
    # Predict the output using the linear model
    y_hat = lm.predict(x)
    # Calculate the error
    error = y_hat - y
    # Compute the weight update term
    delta_w = (error * x).reshape(-1, 1)
    # Update the weights using the learning rate
    lm.W -= sgd.alpha * (delta_w + (lm.lambda_ * lm.W))
    # END SOLUTION




In [81]:
# Test cell for the update function for l2 regularized linear regression. Don't change this cell.
test_rlr_inputs = 3
test_rlr_tolerance = 1e-2
test_rlr = RegularizedModel(test_rlr_inputs, 1, 0.1)
test_rlr_sgd = SGD(0.01)

# First test
rlr_x = np.arange(1, test_rlr_inputs + 1)
rlr_y = np.array([1])
#print(rlr_x, rlr_y, test_rlr.W)
update(test_rlr, test_rlr_sgd, rlr_x, rlr_y)
#print(rlr_x, rlr_y, test_rlr.W)

# Second test
rlr_x = (np.arange(1, test_rlr_inputs + 1) + 1) / 2
rlr_y = np.array([2])
#print("here",rlr_x, rlr_y, test_rlr.W)
update(test_rlr, test_rlr_sgd, rlr_x, rlr_y)
#print("afeter",rlr_x, rlr_y, test_rlr.W)

# Third test
rlr_x = (np.arange(1, test_rlr_inputs + 1) + 2) / 3
rlr_y = np.array([3])
update(test_rlr, test_rlr_sgd, rlr_x, rlr_y)
#print("after",rlr_x, rlr_y, test_rlr.W)
#print(test_rlr.W.shape)

# Check if the weights are approximately correct
the_test = np.allclose(
    test_rlr.W,
    np.array([[0.056891877],[0.085672676], [0.114453474]]),
    atol=test_rlr_tolerance
)

assert the_test, "Weights not updated properly: ❌"
print(f"Test passed: ✅")


Test passed: ✅


Now we train a regularized model with $\lambda = 1.0$ for the synthetic dataset using SGD with $\alpha = 0.01$.


In [82]:
def run_RLR_model(X, Y, num_epochs=100):

    # Construct the model
    sgd = SGD(0.01)  # Learning rate of 0.01
    model = RegularizedModel(X.shape[1], 1, 1.0)  # Create a model with input dimension X.shape[1]

    # Each call to train performs a single epoch
    for _ in range(num_epochs):
      train(model, sgd, X, Y)

    return model

In [83]:
# Train the linear model
RLR_model = run_RLR_model(trainX, trainY, num_epochs=10)

# Compute RMSE on the test data
predictions = RLR_model.predict(testX)
print("prediction shape",predictions.shape)

rmse_RLR = RMSE(predictions, testY)
print("RMSE:", rmse_RLR)


prediction shape (5000, 1)
(5000, 1)
(5000, 1)
RMSE: 1.1927765185419752


# Extra - Checkpoint: RLR better than LR?

Now we get to revisit our synthetic data generator and ask: what data is more suitable for the assumptions underlying L2 regularized linear regression?

Go back to your simulated data generator and modify $f(X)$ so that RLR regression finds a better approximation than vanilla linear regression. Specifically, define a function $f(x)$ such that

Hint: think about what restrictions l2 regularization places on the approximation.

## Remember to return $f(X)$ to the older version before continuing. This is to ensure everything will run as expected.

In [46]:
checkpoint  = rmse_linear < rmse_RLR
if checkpoint:
   print("RMSE LR < RMSE RLR: ❌")
else:
   print(f"Test passed: ✅")

100.74353115425284 100.47846480210892
Test passed: ✅


# **ADAM Optimizer**

In the last section, we implemented simple linear regression with stochastic gradient descent updates using a fixed stepsize. However, it is common to use more sophisticated optimization techniques such as ADAM which adapt the stepsize based on information from consecutive updates. In this section, we will extend our linear regression implementation to also allow the use of the ADAM optimizer.

The original ADAM paper can be found here: https://arxiv.org/pdf/1412.6980. The notes also provide an explanation and pseudocode for the ADAM optimizer.


## ADAM Class

First, we need to define a class that specifies the hyperparameters for ADAM. ADAM also has a state that must be maintained (i.e. $m_t$ and $v_t$), and we will store these on the object as well. 

The ADAM update rule is as follows. Update the exponential moving averages of the gradient and squared gradient, in $m$ and $v$ respectively.

$$
m_{t} = \beta_1 m_{t-1} + (1 - \beta_1) \nabla_w f(w)
$$

$$
v_{t} = \beta_2 v_{t-1} + (1 - \beta_2) (\nabla_w f(w))^2
$$

Then make sure to perform *initialization bias correction* to adjust for the fact that we initialize the moving averages to zero.

$$
\hat{m}_{t} = \frac{m_t}{1-\beta_1^t}
$$

$$
\hat{v}_{t} = \frac{v_t}{1-\beta_2^t}
$$

Finally, we update the model weights.

$$
w_{t} = w_{t-1} - \alpha \frac{m_{t}}{\sqrt{v_{t}} + \epsilon}
$$

where the square for $v_t$ is element-wise. Both $\beta_1$, $\beta_2$, and $\epsilon$ are new hyperparameters for ADAM, and $\alpha$ is still a consistent global stepsize.


In [84]:
class ADAM:
    def __init__(self, alpha: float, beta1: float, beta2: float, in_dim: int, out_dim: int):
        self.alpha: float = alpha
        self.beta1: float = beta1
        self.beta2: float = beta2
        self.epsilon: float = 1e-8
        self.m: np.ndarray = np.zeros((in_dim, out_dim))
        self.v: np.ndarray = np.zeros((in_dim, out_dim))
        self.t: int = 0  # for bias correction


One nice thing about ADAM is that it does not require changes to our `train` function that we've already implemented.  
We still need to loop through each element in the dataset in a random order, then call the `update` function with one data-point at a time.

So all we need to implement now is the new `update` function.  
We've started you off with the function signature.

**Hint:** Don't forget that elementwise operations in Python are done using NumPy functions or array broadcasting. For example:

```python
import numpy as np
x = np.ones(10)
elementwise_sqrt = np.sqrt(x)
elementwise_add = x + 1


## ADAM Update

Implement the ADAM update in `update` below.

We have also provided a test cell that you can use to test your code. Once again, do not change the code in this test cell as doing so will result in point reductions.


In [85]:
def update(lm, opt, x, y):
    # BEGIN SOLUTION
    # Prediction
    y_hat = lm.predict(x)
    # Reshape y_hat to a row vector if necessary
    if y_hat.shape[0] == 1:
        y_hat = y_hat.reshape(-1)
    # Compute error
    error = y_hat - y
    # Compute the weight update
    delta_w = (error * x).reshape(-1, 1)
    # Update time step
    opt.t += 1
    # Update moving averages for gradient (m) and squared gradient (v)
    opt.m = opt.beta1 * opt.m + (1 - opt.beta1) * delta_w
    opt.v = opt.beta2 * opt.v + (1 - opt.beta2) * delta_w**2
    # Bias correction
    m_hat = opt.m / (1 - opt.beta1**opt.t)
    v_hat = opt.v / (1 - opt.beta2**opt.t)
    # Update the weights
    lm.W -= (opt.alpha * m_hat) / (np.sqrt(v_hat) + opt.epsilon)
    # END SOLUTION


In [86]:
# Test cell for the update function for regularized linear regression. Don't change this cell.
test_adam_inputs = 3
test_adam_tolerance = 1e-5

# Create a LinearModel and an ADAM optimizer
test_adam_model = LinearModel(test_adam_inputs, 1)
test_adam = ADAM(alpha=0.01, beta1=0.9, beta2=0.999, in_dim=test_adam_inputs, out_dim=1)

# First update with (1, 1)
test_adam_x = np.array([1, 2, 3])
test_adam_y = np.array([1])
update(test_adam_model, test_adam, test_adam_x, test_adam_y)

# Second update with (0.5, -2)
test_adam_x = (np.array([1, 2, 3]) + 1) / 2
test_adam_y = np.array([-2])
update(test_adam_model, test_adam, test_adam_x, test_adam_y)

# Third update with (0.666..., 3)
test_adam_x = (np.array([1, 2, 3]) + 2) / 3
test_adam_y = np.array([3])
update(test_adam_model, test_adam, test_adam_x, test_adam_y)

# Check if the updated weights are approximately equal to the expected values
expected_weights = np.array([[0.009569011794190506], [0.010764570223018575], [0.011296195526636793]])

# Use np.allclose for approximate comparison with tolerance
test =  np.allclose(test_adam_model.W, expected_weights, atol=test_adam_tolerance)

# Run the test
assert test, "Weights not updated properly: ❌"
print(f"Test passed: ✅")

Test passed: ✅


### Checkpoint: Learning with Adam
It is useful to know how to implement ADAM. For this assignment, however, it likely will not perform better than SGD, because of how our synthetic data is generated. The inputs are reasonably scaled, so we are unlikely to have very different curvature in different dimensions. But, let's run it anyway and check!



In [88]:
# Define the number of epochs
NUM_EPOCHS = 10  # You can adjust this according to your use case

# Function to run a Linear Model with ADAM optimizer
def run_linear_model_with_adam(X, Y):
    # Initialize the ADAM optimizer with (stepsize, beta1, beta2, features, outputs)
    adam = ADAM(alpha=0.01, beta1=0.9, beta2=0.999, in_dim=X.shape[1], out_dim=1)

    # Initialize a Linear Model
    model = LinearModel(X.shape[1], 1)

    # Train the model for a number of epochs
    for _ in range(NUM_EPOCHS):
        train(model, adam, X, Y)

    return model

# Run the linear model with ADAM on training data
__linear_adam_model = run_linear_model_with_adam(trainX, trainY)
rmse_linear_adam = RMSE(__linear_adam_model.predict(testX), testY)
print("RMSE with ADAM:", rmse_linear_adam)

check = np.isclose(rmse_linear, rmse_linear_adam, atol=0.1)
assert check, "ADAM and SGD to different: ❌"
print(f"Test passed: ✅")


(5000, 1)
(5000, 1)
RMSE with ADAM: 1.0121993169655763
Test passed: ✅


# **Multinomial Logistic Regression**
In this section of the notebook, we will be implementing another GLM: multinomial logistic regression. As with the LR implementation, we will start by constructing a synthetic dataset.


### Synthetic Data

Now we will generate a synthetic dataset for multinomial logistic regression (MLM). The only thing that changes is $p(y | \mathbf{x})$, the distribution over $y$. Again, we do not make particular assumptions about the distribution over $\mathbf{x}$. Therefore, we will simply keep the same data matrix `X`.
And now we have different probabilistic assumptions on our response matrix, `Y`, which we need our synthetic dataset to meet.
We will be doing 4-class classification, so our response matrix, `Y`, needs to be of shape `(samples, 4)`.

This time we will specify most of the data-generating function, but we will get you to help us with part of the implementation. We need your help implementing the softmax and helping us generate labels using our data-generating function. Together, we implement a function `g(x)` that produces a vector $y \in [0,1]^4$ for each sample `x`, where we ensure that we have class balance. In other words, the dataset has a roughly even number of samples per class, with at least 3000 of each class present. Further, we ensure that `g(x)` induces a valid Multinomial distribution over the 4 classes. Therefore, $y = g(x)$ should sum to 1 for each input vector `x`.




Before we continue to implement `g(x)`, we need a helper `softmax` function. We need your help to implement that now.

In [89]:
def softmax(y):

    # BEGIN SOLUTION
    return np.exp(y)/np.sum(np.exp(y))
    # END SOLUTION


In [90]:
def test_softmax():
    softmax_result = softmax(np.array([1, 2, 3, 4]))
    expected_result = [0.03205860328008499, 0.08714431874203257, 0.23688281808991013, 0.6439142598879722]

    return np.allclose(softmax_result, expected_result, atol=1e-5)

__check_softmax = test_softmax()
assert __check_softmax, "Softmax not implemented properly: ❌"
print(f"Test passed: ✅")

Test passed: ✅


Now here is our `g(x)` implementation.

In [91]:
def generate_probabilities(X, rng):
    num_features = X.shape[1]  # Assuming X is a 2D array (samples, features)

    # Initialize W with random values drawn from a normal distribution
    W = rng.normal(0, 0.5, size=(num_features, 4))

    # Manually set certain rows of W to 0
    W[0:4, 0] = 0
    W[4:8, 1] = 0
    W[8:12, 2] = 0
    W[12:15, 3] = 0

    # Calculate Y as X * W
    Y = np.dot(X, W)

    # Apply softmax to each row of Y
    for i in range(Y.shape[0]):
        Y[i] = softmax(Y[i])

    return Y



In [92]:
def generate_labels(X, rng):

    num_samples, num_features = X.shape
    glmY = np.zeros((num_samples, 4))  # Response matrix
    probs = generate_probabilities(X, rng)  # Probabilities for each class

    # Ensure at least 3000 samples per class
    target_class_count = 3000
    class_samples = np.zeros(4)

    for idx in range(num_samples):
        # Generate labels using the multinomial distribution
        label = rng.multinomial(1, probs[idx, :])
        glmY[idx, :] = label
        class_samples += label

    return glmY, probs

Now we need to sample a response matrix `Y` according to the probabilities produced by `g(x)`. We will test a few properties of your resulting response matrix and probability matrix. Again do not change any of these test. The tests are below:
-  We expect at least 3000 of each class to be present.
-  We expect `y = g(x)` to sum to 1 for all values of `x`.
-  We expect none of `y` to be a one-hot vector.
-  We expect none of the probabilities to be exactly uniform.


In [93]:
# Simulating the generation of labels and probabilities
rng = np.random.default_rng(seed=44)

glmY, glmProbs = generate_labels(X, rng)
print(glmY.shape)
print(glmProbs.shape)
print(glmY)
print(glmProbs)

# Checks
__check_syn_mn_3000 = np.all(np.count_nonzero(glmY == 1, axis=0) >= 3000)
__check_syn_mn_probs = np.all(np.abs(np.sum(glmProbs, axis=1) - 1) < 1e-6)
__check_syn_mn_onehot = np.all(np.all(glmProbs != 1, axis=1))
__check_syn_mn_uniform = not np.any([np.all(np.abs(r - 1/len(r)) < 1e-6) for r in glmProbs])


# Output the results
print(f"Now we need to sample a response matrix `Y` according to the probabilities produced by `g(x)`. We will test a few properties of your resulting response matrix and probability matrix. Again do not change any of these test. The tests are below:")

assert __check_syn_mn_3000, "We expect at least 3000 of each class to be present: ❌"
print(f"Number of samples in each class bigger than 3000: ✅")

assert __check_syn_mn_probs, "We expect y = g(x) to sum to 1 for all values of x: ❌"
print(f"y sums to 1: ✅")

assert __check_syn_mn_onehot, "We expect none of y to be a one-hot vector: ❌"
print(f"y is a one-hot vector: ✅")

assert __check_syn_mn_uniform, "We expect none of the probabilities to be exactly uniform: ❌"
print(f"Probabilities are uniform: ✅")


(30000, 4)
(30000, 4)
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
[[0.28248407 0.29554503 0.30089064 0.12108027]
 [0.22192719 0.29274442 0.33966657 0.14566181]
 [0.00765409 0.01606963 0.01326706 0.96300921]
 ...
 [0.2343016  0.13249829 0.16156274 0.47163737]
 [0.23860221 0.20205759 0.36956825 0.18977194]
 [0.21938249 0.30636281 0.28624325 0.18801144]]
Now we need to sample a response matrix `Y` according to the probabilities produced by `g(x)`. We will test a few properties of your resulting response matrix and probability matrix. Again do not change any of these test. The tests are below:
Number of samples in each class bigger than 3000: ✅
y sums to 1: ✅
y is a one-hot vector: ✅
Probabilities are uniform: ✅


## Estimator

Like before we start by defining the struct and the prediction functions. Except this time you need to implement the prediction functions for the `MultinomialLogisticModel`. You can use the `softmax` functions defined for matrices and vectors.

In [94]:
class MultinomialLogisticModel:
    def __init__(self, input_dim: int, output_dim: int):
        # Initialize the weights with zeros (input_dim x output_dim)
        self.W = np.zeros((input_dim, output_dim))


In [100]:
def predict(m, X):
    # BEGIN SOLUTION
    return softmax(X @ m.W)
    # END SOLUTION

In [101]:
# Check the vector prediction case
test_mlm = MultinomialLogisticModel(10, 3)
rng = np.random.default_rng(10)
test_mlm.W = rng.standard_normal(test_mlm.W.shape)

# Generate prediction
y_hat = predict(test_mlm, rng.standard_normal(10))
print("y_hat", y_hat)

# Check vector type and values
check_vec_type = isinstance(y_hat, np.ndarray) and y_hat.ndim == 1
check_vec_val = np.allclose(y_hat, np.array([0.08541809, 0.87263002, 0.04195189]))

print("Test 1:")
assert check_vec_type, "y_hat with wrong dimension: ❌"
assert check_vec_type, "y_hat with wrong values: ❌"
print("Test 1 passed: ✅")


# Check the matrix prediction case
Y_hat = predict(test_mlm, rng.standard_normal((8, 10)))
# Define the true matrix values
print("Y_hat", Y_hat)
true_Y = np.array([
    [2.78519680e-03, 3.05590648e-03, 6.69392612e-05],
    [4.19245799e-02, 2.90392619e-04, 1.76737668e-01],
    [8.64826551e-02, 9.46147787e-03, 4.40367730e-01],
    [1.79882209e-03, 1.28421775e-04, 1.97600776e-03],
    [2.99618883e-03, 1.61422837e-01, 5.96514115e-05],
    [9.70045874e-03, 1.88013696e-04, 7.30245474e-03],
    [4.67462451e-04, 2.85263534e-03, 3.02420096e-03],
    [4.33722567e-02, 6.87386049e-05, 3.46930412e-03]])


# Check matrix type and values
check_mat_type = isinstance(Y_hat, np.ndarray) and Y_hat.ndim == 2
check_mat_val = np.allclose(Y_hat, true_Y)

print("Test 2:")
assert check_mat_type, "Y_hat with wrong dimension: ❌"
assert check_mat_val, "Y_hat with wrong values: ❌"
print("Test 2 passed: ✅")

y_hat [0.08541809 0.87263002 0.04195189]
Test 1:
Test 1 passed: ✅
Y_hat [[2.78519680e-03 3.05590648e-03 6.69392612e-05]
 [4.19245799e-02 2.90392619e-04 1.76737668e-01]
 [8.64826551e-02 9.46147787e-03 4.40367730e-01]
 [1.79882209e-03 1.28421775e-04 1.97600776e-03]
 [2.99618883e-03 1.61422837e-01 5.96514115e-05]
 [9.70045874e-03 1.88013696e-04 7.30245474e-03]
 [4.67462451e-04 2.85263534e-03 3.02420096e-03]
 [4.33722567e-02 6.87386049e-05 3.46930412e-03]]
Test 2:
Test 2 passed: ✅


In [102]:
def update(mlm, opt, x, y):
    #### BEGIN SOLUTION
    # Predict output
    y_hat = predict(mlm, x)
    # Compute error
    error = y_hat - y
    # Gradient of the weights (ΔW), outer product of x and err
    delta_W = np.outer(x, error)
    # Update time step
    opt.t += 1
    # Update first moment estimate (m)
    opt.m = opt.beta1 * opt.m + (1 - opt.beta1) * delta_W
    # Update second moment estimate (v)
    opt.v = opt.beta2 * opt.v + (1 - opt.beta2) * delta_W**2
    # Correct bias for first and second moments
    m_hat = opt.m / (1 - opt.beta1**opt.t)
    v_hat = opt.v / (1 - opt.beta2**opt.t)
    # Update weights with the Adam optimizer step
    mlm.W -= opt.alpha * m_hat / (np.sqrt(v_hat) + opt.epsilon)
    #### END SOLUTION

In [103]:
# Test MLM Update
test_mlm_inputs = 3
test_mlm_outputs = 4
test_mlm_tolerance = 1e-3  # Absolute tolerance to consider arrays equal
test_mlm = MultinomialLogisticModel(test_mlm_inputs, test_mlm_outputs)

# Pass the correct dimensions to ADAM
test_mlm_adam = ADAM(0.1, 0.9, 0.999, test_mlm_inputs, test_mlm_outputs)

# Update 1
test_mlm_x = (np.arange(1, test_mlm_inputs + 1) - (test_mlm_inputs // 2)) / 2
test_mlm_y = np.zeros(test_mlm_outputs)
test_mlm_y[test_mlm_outputs // 2] = 1
update(test_mlm, test_mlm_adam, test_mlm_x, test_mlm_y)

# Update 2
test_mlm_x = np.arange(1, test_mlm_inputs + 1) / 5
test_mlm_y = np.zeros(test_mlm_outputs)
test_mlm_y[-1] = 1
update(test_mlm, test_mlm_adam, test_mlm_x, test_mlm_y)

# Update 3
test_mlm_x = -np.arange(1, test_mlm_inputs + 1)
test_mlm_y = np.zeros(test_mlm_outputs)
test_mlm_y[0] = 1
update(test_mlm, test_mlm_adam, test_mlm_x, test_mlm_y)


# Expected weight matrix after updates
expected = np.array([ [-0.14249884, -0.01810744, -0.06248441,  0.16037613],
                      [-0.27128431, -0.15108453,  0.19392424,  0.01265325],
                      [-0.26940328, -0.15097165,  0.2012634,  -0.00388091]]
                      )

# Check if the weights are approximately equal to the expected values
test = np.allclose(test_mlm.W, expected, atol=test_mlm_tolerance)
assert test, "Wrong weights: ❌"
print("Test passed: ✅")


Test passed: ✅


In [104]:
# Define the train and test labels as before
print(glmY.shape)
glm_trainY = glmY[:25000, :]
glm_testY = glmY[25000:]


(30000, 4)


## Checkpoint: Classification

First, we learn a `MultinomialLogisticModel` and then a Logistic Regression (LR) model and test. We should see that the `MultinomialLogisticModel` performs better!

We use LR in the most naive way: we predict the target vector. This means LR is trying to predict 0s and 1s. The output from LR is not guaranteed to be an indicator vector; in fact, it's unlikely to be. We return the class with the highest output from LR.

For example, if LR outputs `[1.3, -0.5, 0.1, -4.2]`, then we say that LR has classified this input as class 1, since 1.3 is the highest.


In [105]:
def run_mlm(X, Y, num_epochs=10):
    adam = ADAM(alpha=0.001, beta1=0.8, beta2=0.999, in_dim=X.shape[1], out_dim=4)

    model = MultinomialLogisticModel(X.shape[1],4)

    for _ in range(num_epochs):
        print(_)
        train(model, adam, X, Y)

    return model


In [106]:
print(trainX.shape) #figure out where glm comes from, which X, and where to set num_samples
print(glm_trainY.shape)
mlm_model = run_mlm(trainX, glm_trainY)
rmse_mlm= RMSE(predict(mlm_model, testX), glm_testY)
print("Rmse", rmse_mlm)

(25000, 16)
(25000, 4)
0
1
2
3
4
5
6
7
8
9
(5000, 4)
(5000, 4)
Rmse 0.49995640935176977


In [107]:
# Define the Linear Model for classification
def run_lr_for_classification(X, Y):
    adam = ADAM(0.001, 0.9, 0.999, X.shape[1], 4)  # Initialize ADAM optimizer
    model = LinearModel(X.shape[1], 4)  # Create the Linear Model

    for _ in range(NUM_EPOCHS):  # Train for NUM_EPOCHS
        train(model, adam, X, Y)

    return model

# Run Linear Model for classification
linear_model_class = run_lr_for_classification(trainX, glm_trainY)

# Compute RMSE on the test data
rmse = RMSE(predict(linear_model_class, testX), glm_testY)

# Define the classification error function
def classification_error(y_hat, y):
    max_y_hat = np.argmax(y_hat, axis=1)
    max_y = np.argmax(y, axis=1)
    return 1 - np.mean(max_y_hat == max_y)

# Calculate classification errors for both models
lm_cerror = classification_error(predict(linear_model_class, testX), glm_testY)

mlm_cerror = classification_error(predict(mlm_model, testX), glm_testY)


print(f"Below we check the classification error using the function `classification_error`. The error for the models are as follows:")
print(f"- `LinearModel`: {lm_cerror}")
print(f"- `MultinomialLogisticModel`: {mlm_cerror}")

# Check which model has a smaller classification error
check_classification_error = lm_cerror > mlm_cerror

print(check_classification_error)



(5000, 4)
(5000, 4)
Below we check the classification error using the function `classification_error`. The error for the models are as follows:
- `LinearModel`: 0.5724
- `MultinomialLogisticModel`: 0.5714
True
